In [1]:
import scipy as sp
import pandas as pd
import numpy as np
import scipy as sp

from konlpy.tag import Kkma       ; kkma = Kkma()
from konlpy.tag import Hannanum   ; hannanum = Hannanum()
from konlpy.tag import Okt        ; t = Okt()
from konlpy.tag import *
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# 평점 전처리 함수
def star_preprocessing(value) :
#     value = int(text)

    if value <= 3 :
        return '0'
    else :
        return '1'

# 형태소 분석을 위한 함수
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)

In [3]:
def step1_data_preprocessing() :
    # 수집한 데이터를 읽어온다.
    df = pd.read_csv('data/cos27.csv')

    # 전처리를 수행한다.
    df['rating'] = df['rating'].apply(star_preprocessing)

    # 학습데이터와 테스트 데이터로 나눈다.
    text_list = df['content'].tolist()
    star_list = df['rating'].tolist()

    from sklearn.model_selection import train_test_split

    # 80%는 학습, 20%는 test
    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.2, random_state=0)

    return text_train, text_test, star_train, star_test


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
from konlpy.tag import *

In [4]:
df = pd.read_csv('data/cos27.csv')

In [5]:
df.shape

(4000, 2)

In [6]:
def step2_learning(X_train, y_train, X_test, y_test):
    # 주어진 데이터를 단어 사전으로 만들고 각 단어의 빈도수를 계산한 후 벡터화 하는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)

    # 문장별 나오는 단어수 세서 수치화, 벡터화해서 학습을 시킨다.
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습한다.
    pipe.fit(X_train, y_train)

    # 학습 정확도 측정
    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))

    # 학습된 모델을 저장한다.
    with open('pipe.dat', 'wb') as fp :
        pickle.dump(pipe, fp)
        
    print('저장완료')

In [7]:
def step3_using_model() :
    # 객체를 복원한다.
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)

    import numpy as np

    while True :
        text = input('리뷰를 작성해주세요 :')

        str = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(str) * 100)
        # 예측 결과
        r2 = pipe.predict(str)[0]

        if r2 == '1' :
            print('긍정적인 리뷰')
        else :
            print('부정적인 리뷰')

        print('정확도 : %.3f' % r1)

        
# 학습 함수
def learing() :
    text_train, text_test, star_train, star_test = step1_data_preprocessing()
    step2_learning(text_train, star_train, text_test, star_test)

# 사용 함수
def using() :
    step3_using_model()

In [ ]:
learing()
using()

C:\Users\runia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.80125
저장완료
리뷰를 작성해주세요 :"샘플 받아서 쓰고 있는데 뭐랄까 은근 겉도는 느낌이랄까요  시간 좀 지나면 흡수되긴 하는데 엄청 촉촉한것두 아니고 씁쓸한?냄새도 별로네영..ㅎㅎㅎ"
부정적인 리뷰
정확도 : 90.611
리뷰를 작성해주세요 :"향은 없어요~ 살짝 묽은 느낌이어서 걱정했는데 촉촉함이 오래가서 너무 좋아요~"
긍정적인 리뷰
정확도 : 67.952
리뷰를 작성해주세요 :"바디로션이지만 얼굴에도 발라본 결과 얼굴이 뽀얘지고 쫀쫀해지는 듯한 느낌을 받았어요!"
긍정적인 리뷰
정확도 : 77.574
리뷰를 작성해주세요 :"너무 고보습이거나 저보습도 아니고 딱 좋아요~ 계절 상관 없이 무난하게 쓸 수 있고 건조함은 못 느꼈습니다. 그리고 용량대비 가격도 저렴해서 아끼지 않고 쓸 수 있네요. 그런데 살 때 몰랐는데 벤질알코올 성분이 있네요. 바디용이라 그냥 쓰긴 했는데 찝찝해서 재구매는 안 하려구요. 온가족이 쓰는 로션이라는 브랜드 이미지가 있었는데 실망스러워요."
긍정적인 리뷰
정확도 : 81.982
리뷰를 작성해주세요 :"유럽에있는동안 찢어질것같은 건조함때문에 내가 지성인걸 잊고살았는데 한국오니까 쓰던게 다 너무 심하게 기름져서 다 내다버리고나니까 토너랑 에센스밖에 없길래 로션하나 샀당 이것도 아직까지 내겐 좀 기름지다. + 몇일써보니 이것도 트러블일으킴....이거 바른날은 꼭 좁쌀이 몇개씩올라오고 잠잠해진 트러블들도 들고일어남"
긍정적인 리뷰
정확도 : 69.249
리뷰를 작성해주세요 :"마몽드 로즈 스킨사용하다가 로션도 써볼까하고 사용했는데.. 역시나 저한테는 무거워요. 볼이랑 턱라인 울긋불긋 ㅠㅠ 수부지라서 피부위에서 흡수안되고 겉돌면 바로 트러블나서ㅠㅠ 이렇게 하나 거르고 가요ㅠㅠ 아쉬웡ㅠㅠ"
부정적인 리뷰
정확도 : 90.679
리뷰를 작성해주세요 :"냄새가 너무 자극적이고 피부에 안맞아서 그런지 더 별로였던 제품ㅠㅠ 재구매의사×"
부정적인 리뷰
정확도 : 90.657
